In [1]:
# Compute the camera calibration matrix and distortion coefficients from chessboard iamges
import numpy as np
import cv2
import glob
import matplotlib.pyplot as plt
%matplotlib qt

def camCalibration():
    # Prepare object points
    objp = np.zeros( (6*9,3), np.float32)
    objp[:,:2] = np.mgrid[0:9,0:6].T.reshape(-1,2)

    # Arrays to store object points and image points from all the images
    objpoints = []
    imgpoints = []
 
    # Make a list of calibration images
    images = glob.glob('./camera_cal/calibration*.jpg')

    # Step through the list and search for chessboard corners
    for fname in images:
        img = cv2.imread(fname)
        gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

        # Find the chessboard corners
        ret, corners = cv2.findChessboardCorners(gray, (9,6),None)

        if ret == True:
            objpoints.append(objp)
            imgpoints.append(corners)
        
    ret, mtx, dist, rvecs, tvecs = cv2.calibrateCamera(objpoints, imgpoints, gray.shape[::-1], None, None)
    
    return ret, mtx, dist

# Apply a distortion correction to raw images
def undistortImg( img, mtx, dist ):
    img_size = (img.shape[1], img.shape[0])
    
    img = cv2.undistort(img, mtx, dist, None, mtx)
    
    return img

# Use color transforms, gradients, etc., to create a thresholded binary image.

def abs_sobel_thresh(gray, orient='x', sobel_kernel=3, thresh=(0, 255)):
    # Apply x or y gradient with the OpenCV Sobel() function
    # and take the absolute value
    if orient == 'x':
        abs_sobel = np.absolute(cv2.Sobel(gray, cv2.CV_64F, 1, 0, ksize=sobel_kernel))
    if orient == 'y':
        abs_sobel = np.absolute(cv2.Sobel(gray, cv2.CV_64F, 0, 1, ksize=sobel_kernel))        
    # Rescale back to 8 bit integer
    scaled_sobel = np.uint8(255*abs_sobel/np.max(abs_sobel))    
    # Create a copy and apply the threshold
    binary_output = np.zeros_like(scaled_sobel)
    # Here I'm using inclusive (>=, <=) thresholds, but exclusive is ok too
    thresh_min = thresh[0]
    thresh_max = thresh[1]
    
    binary_output[(scaled_sobel >= thresh_min) & (scaled_sobel <= thresh_max)] = 1

    # Return the result
    return binary_output

def mag_thresh(gray, sobel_kernel=3, mag_thresh=(0, 255)):    
    # Take both Sobel x and y gradients
    sobelx = cv2.Sobel(gray, cv2.CV_64F, 1, 0, ksize=sobel_kernel)
    sobely = cv2.Sobel(gray, cv2.CV_64F, 0, 1, ksize=sobel_kernel)
    # Calculate the gradient magnitude
    gradmag = np.sqrt(sobelx**2 + sobely**2)
    # Rescale to 8 bit
    scale_factor = np.max(gradmag)/255 
    gradmag = (gradmag/scale_factor).astype(np.uint8) 
    # Create a binary image of ones where threshold is met, zeros otherwise
    binary_output = np.zeros_like(gradmag)
    binary_output[(gradmag >= mag_thresh[0]) & (gradmag <= mag_thresh[1])] = 1

    # Return the binary image
    return binary_output

def dir_threshold(gray, sobel_kernel=3, thresh=(0, np.pi/2)):
    # Calculate the x and y gradients
    sobelx = cv2.Sobel(gray, cv2.CV_64F, 1, 0, ksize=sobel_kernel)
    sobely = cv2.Sobel(gray, cv2.CV_64F, 0, 1, ksize=sobel_kernel)
    # Take the absolute value of the gradient direction, 
    # apply a threshold, and create a binary image result
    absgraddir = np.arctan2(np.absolute(sobely), np.absolute(sobelx))
    binary_output =  np.zeros_like(absgraddir)
    binary_output[(absgraddir >= thresh[0]) & (absgraddir <= thresh[1])] = 1

    # Return the binary image
    return binary_output


# Color space
def hls_select_s(img, thresh=(0, 255)):
    hls = cv2.cvtColor(img, cv2.COLOR_RGB2HLS)
    s_channel = hls[:,:,2]
#     binary_output = np.zeros_like(s_channel)
#     binary_output[(s_channel > thresh[0]) & (s_channel <= thresh[1])] = 1
    binary_output = s_channel
    return binary_output

def hls_select_h(img, thresh=(0, 255)):
    hls = cv2.cvtColor(img, cv2.COLOR_RGB2HLS)
    s_channel = hls[:,:,1]
    binary_output = np.zeros_like(s_channel)
    binary_output[(s_channel > thresh[0]) & (s_channel <= thresh[1])] = 1   
    return binary_output

# Apply a perspective transform to rectify binary image ("birds-eye view").
def convBirdEyeView(img, src, dst ):
#     img_size = (img.shape[1], img.shape[0])
    img_size = (480, 720)

    M = cv2.getPerspectiveTransform(src, dst)
    Minv = cv2.getPerspectiveTransform(dst, src)
    wrap_img = cv2.warpPerspective(img, M, img_size, flags=cv2.INTER_LINEAR)

    return wrap_img, Minv

# Detect lane pixels and fit to find the lane boundary.
def findLane(img, bCRegion = False ):
    # Choose the number of sliding windows
    nwindows = 15
    
    # Assuming you have created a warped binary image called "binary_warped"
    # Take a histogram of the bottom half of the image
    dist = list( range(int(img.shape[0]/2), img.shape[0]))    
    histogram = np.dot( dist, img[img.shape[0]/2:,:])  
    # Create an output image to draw on and  visualize the result
    out_img = np.dstack((img, img, img))  * 255  
    
    # Find the peak of the left and right halves of the histogram
    # These will be the starting point for the left and right lines
    midpoint = np.int(histogram.shape[0]/2)
    leftx_base = np.argmax(histogram[:midpoint])
    rightx_base = np.argmax(histogram[midpoint:]) + midpoint    
    # Set height of windows
    window_height = np.int(img.shape[0]/nwindows)
    # Identify the x and y positions of all nonzero pixels in the image
    nonzero = img.nonzero()
    nonzeroy = np.array(nonzero[0])
    nonzerox = np.array(nonzero[1])
    # Current positions to be updated for each window
    leftx_current = leftx_base
    rightx_current = rightx_base
    # Set the width of the windows +/- margin
    margin = 45
    # Set minimum number of pixels found to recenter window
    minpix = 20
    # Create empty lists to receive left and right lane pixel indices
    left_lane_inds = []
    right_lane_inds = []

    # Step through the windows one by one
    for window in range(nwindows):
        # Identify window boundaries in x and y (and right and left)
        win_y_low = img.shape[0] - (window+1)*window_height
        win_y_high = img.shape[0] - window*window_height
        win_xleft_low = leftx_current - margin
        win_xleft_high = leftx_current + margin
        win_xright_low = rightx_current - margin
        win_xright_high = rightx_current + margin
        # Draw the windows on the visualization image
        cv2.rectangle(out_img,(win_xleft_low,win_y_low),(win_xleft_high,win_y_high),(0,255,0), 2) 
        cv2.rectangle(out_img,(win_xright_low,win_y_low),(win_xright_high,win_y_high),(0,255,0), 2) 
        # Identify the nonzero pixels in x and y within the window
        good_left_inds = ((nonzeroy >= win_y_low) & (nonzeroy < win_y_high) & (nonzerox >= win_xleft_low) & (nonzerox < win_xleft_high)).nonzero()[0]
        good_right_inds = ((nonzeroy >= win_y_low) & (nonzeroy < win_y_high) & (nonzerox >= win_xright_low) & (nonzerox < win_xright_high)).nonzero()[0]
        # Append these indices to the lists
        left_lane_inds.append(good_left_inds)
        right_lane_inds.append(good_right_inds)
        # If you found > minpix pixels, recenter next window on their mean position
        if len(good_left_inds) > minpix:
            leftx_current = np.int(np.mean(nonzerox[good_left_inds]))
        if len(good_right_inds) > minpix:        
            rightx_current = np.int(np.mean(nonzerox[good_right_inds]))

    # Concatenate the arrays of indices
    left_lane_inds = np.concatenate(left_lane_inds)
    right_lane_inds = np.concatenate(right_lane_inds)
    
    # Extract left and right line pixel positions
    leftx = nonzerox[left_lane_inds]
    lefty = nonzeroy[left_lane_inds] 
    rightx = nonzerox[right_lane_inds]
    righty = nonzeroy[right_lane_inds] 
    
    # Fit a second order polynomial to each
    left_fit = np.polyfit(lefty, leftx, 2)
    right_fit = np.polyfit(righty, rightx, 2)
    
    # Generate x and y values for plotting
    ploty = np.linspace(0, img.shape[0]-1, img.shape[0] )    
    left_fitx = left_fit[0]*ploty**2 + left_fit[1]*ploty + left_fit[2]
    right_fitx = right_fit[0]*ploty**2 + right_fit[1]*ploty + right_fit[2]

    out_img[nonzeroy[left_lane_inds], nonzerox[left_lane_inds]] = [255, 0, 0]
    out_img[nonzeroy[right_lane_inds], nonzerox[right_lane_inds]] = [0, 0, 255]

    if bCRegion == True:
        # Generate a polygon to illustrate the search window area
        # And recast the x and y points into usable format for cv2.fillPoly()
        left_line_window1 = np.array([np.transpose(np.vstack([left_fitx-margin, ploty]))])
        left_line_window2 = np.array([np.flipud(np.transpose(np.vstack([left_fitx+margin, ploty])))])
        left_line_pts = np.hstack((left_line_window1, left_line_window2))
        right_line_window1 = np.array([np.transpose(np.vstack([right_fitx-margin, ploty]))])
        right_line_window2 = np.array([np.flipud(np.transpose(np.vstack([right_fitx+margin, ploty])))])
        right_line_pts = np.hstack((right_line_window1, right_line_window2))

        # Draw the lane onto the warped blank image
        cv2.fillPoly(window_img, np.int_([left_line_pts]), (0,255, 0))
        cv2.fillPoly(window_img, np.int_([right_line_pts]), (0,255, 0))
        out_img = cv2.addWeighted(out_img, 1, window_img, 0.3, 0)        

    # Calculate a curvature
    y_eval = 0
    
    ym_per_pix = 30/720
    xm_per_pix = 3.7/310
    
    # Fit new polynomials to x,y in world space    
    fit_cr_left = np.polyfit(ploty*ym_per_pix, (left_fitx[::-1])*xm_per_pix, 2)
    fit_cr_right = np.polyfit(ploty*ym_per_pix, (right_fitx[::-1])*xm_per_pix, 2)
    
    # Calculate the new radii of curvature
    curverad_left = ((1 + (2*fit_cr_left[0]*y_eval*ym_per_pix + fit_cr_left[1])**2)**1.5) / np.absolute(2*fit_cr_left[0])
    curverad_right = ((1 + (2*fit_cr_right[0]*y_eval*ym_per_pix + fit_cr_right[1])**2)**1.5) / np.absolute(2*fit_cr_right[0])
    
    
    # Offset
    pos_left = left_fit[0]*y_eval**2 + left_fit[1]*y_eval + left_fit[2]
    pos_right = right_fit[0]*y_eval**2 + right_fit[1]*y_eval + right_fit[2]    
    midpoint = (pos_left + pos_right)/2
    offset = (midpoint - img.shape[1]/2)
    offset_m = offset * xm_per_pix
    
    
    # Draw     
    pts_l = np.transpose( np.vstack( [left_fitx, ploty] ) )
    pts_r = np.transpose( np.vstack( [right_fitx, ploty] ) )    
    cv2.polylines( out_img, np.int_([pts_l]), 0, (0,255,0), thickness=3)
    cv2.polylines( out_img, np.int_([pts_r]), 0, (0,255,0), thickness=3)
    
    return out_img, ploty, left_fitx, right_fitx, curverad_left, curverad_right, offset_m

# Determine the curvature of the lane and vehicle position with respect to center.
class LIne():
    def __init__(self):
        # was the line detected in the last iteration?
        self. detected = False
        # x values of the last n fits of the line
        self.recent_xfitted = []
        # average x values of the fitted line over the last 
        self.bestx = None
        # polynomial coefficients averaged over the last n iterations
        self.best_fit = None
        #polynomial coefficients for the most recent fit
        self.current_fit = [np.array([False])]
        #radius of curvature of the line in som units
        self.radius_of_curvature = None
        #distance in meters of vehicle center frme the line
        self.line_base_pos = None
        #difference in fit coefficients between last and new fits
        self.diffs = np.array([0,0,0], dtype='float')
        #x values for detected ine pixels
        self.allx = None
        #y values for detected line pixels
        self.ally = None
        
        
# Output visual display of the lane boundaries and numerical estimation of lane curvature and vehicle position.
def drawResult( undist, Minv, warped, ploty, left_fitx, right_fitx ):
    # Create an image to draw the lines on
    warp_zero = np.zeros_like(warped).astype(np.uint8)
    color_warp = np.dstack((warp_zero, warp_zero, warp_zero))
    
    # Recast the x and y points into usable format for cv2.fill
    pts_left = np.array( [np.transpose(np.vstack([left_fitx, ploty]))])
    pts_right = np.array( [np.flipud(np.transpose(np.vstack([right_fitx, ploty])))])
    pts = np.hstack( (pts_left, pts_right))
    
    # Draw the lane onto the warped blank image
    cv2.fillPoly( color_warp, np.int_([pts]), (0,255,0))
    
    # Warp the blank back to original image space using inverse perspective matrix (Minv)
    newwarp = cv2.warpPerspective( color_warp, Minv, (undist.shape[1], undist.shape[0]))
    # Combine the result with the orginal image
    result = cv2.addWeighted( undist, 1, newwarp, 0.3, 0)
    
    
    return result
        

In [2]:
import matplotlib.image as mpimg
import numpy as np
import cv2


In [3]:
def lanefilter(img, 
                 Ksizex, threshx, 
                 Ksizey, threshy, 
                 Ksize_mag, thresh_mag, 
                 ksize_dir, thresh_dir):
    
    gradx = abs_sobel_thresh(img, orient='x', sobel_kernel=Ksizex, thresh=(threshx[0], threshx[1]) )
    grady = abs_sobel_thresh(img, orient='y', sobel_kernel=Ksizey, thresh=(threshy[0], threshy[1]) )
    mag = mag_thresh(img, sobel_kernel=Ksize_mag, mag_thresh=(thresh_mag[0], thresh_mag[1]))
    dir = dir_threshold(img, sobel_kernel=ksize_dir, thresh=(thresh_dir[0], thresh_dir[1]))
    
    combined_sobel = np.zeros_like(img)
    combined_dir = np.zeros_like(img)
    combined = np.zeros_like(img)    
    
    combined_sobel[(gradx == 1) | (grady == 1) ] =1
    combined_dir[(mag == 1) & (dir == 1) ] =1    
    combined[(combined_sobel==1)| (combined_dir==1)] =1
    
    return combined_sobel, combined_dir, combined

In [4]:
def lanefilter_angle(img, 
                 Ksizex, threshx, 
                 Ksizey, threshy,                  
                 ksize_dir, thresh_dir):
    
    gradx = abs_sobel_thresh(img, orient='x', sobel_kernel=Ksizex, thresh=(threshx[0], threshx[1]) )
    grady = abs_sobel_thresh(img, orient='y', sobel_kernel=Ksizey, thresh=(threshy[0], threshy[1]) )    
    dir = dir_threshold(img, sobel_kernel=ksize_dir, thresh=(thresh_dir[0], thresh_dir[1]))
    
    combined_sobel = np.zeros_like(img)
    combined_dir = dir
    combined = np.zeros_like(img)    
    
    combined_sobel[(gradx == 1) | (grady == 1) ] =1
    combined[(combined_sobel==1)& (combined_dir==1)] =1
    
    return combined_sobel, combined_dir, combined

In [36]:
def process_image(img, bPlot = False):
    
    # 2. Processing
    # 2.1 Undistortion
    undis_img = undistortImg( img, mtx, dist )
    
    # 2.2 Filtering using gradient (sobel)
    img_hls =  cv2.cvtColor(img, cv2.COLOR_RGB2HLS)
    
    gray = undis_img[:,:,0]
    
    img_r = undis_img[:,:,0]
    img_g = undis_img[:,:,1]
    img_b = undis_img[:,:,2]
    
    img_s = img_hls[:,:,2]
    img_h = img_hls[:,:,0]
    img_l = img_hls[:,:,1]
    
    
    combined_sobel_g, combined_dir_g, combined_g = lanefilter_angle(gray, 
                                                             Ksizex=15, threshx=(5,255), 
                                                             Ksizey=15, threshy=(50,255),                                                              
                                                             ksize_dir=15, thresh_dir=(0.8, 1.4))
    
    combined_sobel_s, combined_dir_s, combined_s = lanefilter(img_s, 
                                                             Ksizex=15, threshx=(30,255), 
                                                             Ksizey=15, threshy=(80,255), 
                                                             Ksize_mag=15, thresh_mag=(40,255), 
                                                             ksize_dir=15, thresh_dir=(0.8, 1.4))
    
    combined_sobel_l, combined_dir_l, combined_l = lanefilter_angle(img_l, 
                                                         Ksizex=15, threshx=(10,255), 
                                                         Ksizey=15, threshy=(30,255),                                                          
                                                         ksize_dir=15, thresh_dir=(0.8, 1.4))
    img_test = np.zeros_like(img_l)
    img_test[(img_h > 100)] = 1 
        
    combined = np.zeros_like(gray)
    combined[ (combined_g==1) | (combined_s==1) ] = 1

    
    offset_x = 100
    offset_y = 100

    #Video 1: project.mp4
#     yframe = 450
#     xframe_ls = 265
#     xframe_rs = 1050
#     xframe_ll = 595
#     xframe_rl = 685
        
#     perImgSize_w = 480
#     perImgSize_h = 720
    

    #Video 2: challenge.mp4
    yframe = 480
    xframe_ls = 300
    xframe_rs = 958    
    xframe_ll = 600
    xframe_rl = 718
        
    perImgSize_w = 480
    perImgSize_h = 720

    src = np.float32( [[xframe_ll, yframe], [xframe_rl, yframe], [xframe_rs, 680], [xframe_ls, 680]] )
    dst = np.float32( [[offset_x,0], [perImgSize_w-offset_x,0], [perImgSize_w-offset_x,perImgSize_h], [offset_x, perImgSize_h]])
    perImg, Minv = convBirdEyeView( combined, src, dst )    
    
    out, ploty, Lfit, Rfit, Curature_left, Curature_right, offset_m = findLane(perImg)
    
    result = drawResult(undis_img, Minv, perImg, ploty, Lfit, Rfit)
    
    font = cv2.FONT_HERSHEY_SIMPLEX
    cv2.putText( result, "Radius Left [m]: " + str(Curature_left), (70, 100),font, 1, (255,0,0), 2, cv2.LINE_AA )    
    cv2.putText( result, "Radius Right[m]: " + str(Curature_right), (70, 150),font, 1, (255,0,0), 2, cv2.LINE_AA )    
    cv2.putText( result, "Offset [m]: " + str(offset_m), (70, 200),font, 1, (255,0,0), 2, cv2.LINE_AA )

    test_img = img
    test_img_per = out
    
    cv2.polylines(test_img, np.int32([src]), 30, (255,0,0))
    cv2.polylines(test_img_per, np.int32([dst]), 30, (255,255,255))
    
    plt.figure(10)
    plt.imshow(test_img)
    plt.figure(11)
    plt.imshow(test_img_per)
    
    if bPlot == True:
        # plot images
        plt.figure(1)
        plt.rcParams["figure.figsize"] = [10, 14]

        ax1 = plt.subplot2grid( (4,3), (0,0) )
        ax1.imshow( combined_sobel_g , cmap='gray')
        ax1.set_title('combined_sobel_g', fontsize=10)

        ax2 = plt.subplot2grid( (4,3), (0,1) )
        ax2.imshow( combined_dir_g , cmap='gray' )
        ax2.set_title('combined_dir_g', fontsize=10)

        ax3 = plt.subplot2grid( (4,3), (0,2) )
        ax3.imshow( combined_g , cmap='gray' )
        ax3.set_title('combined_g', fontsize=10)

        ax4 = plt.subplot2grid( (4,3), (1,0) )
        ax4.imshow( combined_sobel_s, cmap='gray' )
        ax4.set_title('combined_sobel_s', fontsize=10)

        ax5 = plt.subplot2grid( (4,3), (1,1) )
        ax5.imshow( combined_dir_s, cmap='gray' )
        ax5.set_title('combined_dir_s', fontsize=10)

        ax6 = plt.subplot2grid( (4,3), (1,2) )
        ax6.imshow( combined_s, cmap='gray' )
        ax6.set_title('combined_s', fontsize=10)

        ax7 = plt.subplot2grid( (4,3), (2,0) )
        ax7.imshow( combined_sobel_l, cmap='gray' )
        ax7.set_title('combined_sobel_l', fontsize=10)

        ax8 = plt.subplot2grid( (4,3), (2,1) )
        ax8.imshow( combined_dir_l, cmap='gray' )
        ax8.set_title('combined_dir_l', fontsize=10)

        ax9 = plt.subplot2grid( (4,3), (2,2) )
        ax9.imshow( combined_l, cmap='gray' )
        ax9.set_title('combined_l', fontsize=10)

        ax10 = plt.subplot2grid( (4,3), (3,0) )
        ax10.imshow( combined, cmap='gray' )
        ax10.set_title('Combined', fontsize=10)

        ax11 = plt.subplot2grid( (4,3), (3,1) )
        ax11.imshow( out )
        ax11.set_title('Output', fontsize=10)

        ax12 = plt.subplot2grid( (4,3), (3,2) )
        ax12.imshow( result )
        ax12.set_title('result', fontsize=10)

        plt.subplots_adjust( left = 0, right = 1, top = 0.9, bottom=0.)
        
    
    return result

## Calibration

In [6]:
# calibration from chessboard images
ret, mtx, dist = camCalibration()

In [7]:
# Show the undistorted image (example)
chess_img_file = './camera_cal/calibration1.jpg'
chess_img = cv2.imread(chess_img_file)

chess_undis = undistortImg( chess_img, mtx, dist)

cv2.imwrite("./writeup_image/distorted image.jpg", chess_img)
cv2.imwrite("./writeup_image/undistorted image.jpg", chess_undis)

True

## Pipeline

Load a test image

In [42]:
import os

images = glob.glob('./test_images/*.jpg')

for fname in images:    
# fname = './test_images/test1.jpg'

    # 1. Load a test image
    img = cv2.imread(fname)
#     result_img = process_image(img, True)

    # 2. Pipeline for lane finding

    ## 2.1 Undistortion
    undis_img = undistortImg( img, mtx, dist )


    ## 2.2 Image filtering    
    # Color channel: R channel, S channel
    gray = undis_img[:,:,1]
    img_hls =  cv2.cvtColor(undis_img, cv2.COLOR_RGB2HLS)

    gray = undis_img[:,:,1] # Gray(R channel)
    img_s = img_hls[:,:,2] # S channel

    # Sobel filter of R and S channel, respectively
    combined_sobel_g, combined_dir_g, combined_g = lanefilter(gray, 
                                                             Ksizex=15, threshx=(30,255), 
                                                             Ksizey=15, threshy=(80,255), 
                                                             Ksize_mag=15, thresh_mag=(40,255), 
                                                             ksize_dir=15, thresh_dir=(0.7, 1.3))

    combined_sobel_s, combined_dir_s, combined_s = lanefilter(img_s, 
                                                             Ksizex=15, threshx=(30,255), 
                                                             Ksizey=15, threshy=(80,255), 
                                                             Ksize_mag=15, thresh_mag=(40,255), 
                                                             ksize_dir=15, thresh_dir=(0.7, 1.1))

    combined = np.zeros_like(gray)
    combined[ (combined_g==1) | (combined_s==1) ] = 1


    ## 2.3 Perspective transform
    offset_x = 100
    offset_y = 100

    yframe = 450
    xframe_ls = 265
    xframe_rs = 1050
    xframe_ll = 595
    xframe_rl = 685

    perImgSize_w = 480
    perImgSize_h = 720

    src = np.float32( [[xframe_ll, yframe], [xframe_rl, yframe], [xframe_rs, 680], [xframe_ls, 680]] )
    dst = np.float32( [[offset_x,0], [perImgSize_w-offset_x,0], [perImgSize_w-offset_x,perImgSize_h], [offset_x, perImgSize_h]])
    perImg, Minv = convBirdEyeView( combined, src, dst )    

    ## 2.4 Find lanes in a test image
    ## ( with Calculate the radius of curvature of lane and the position of the vehicle )
    out, ploty, Lfit, Rfit, Curature_left,  Curature_right, offset_m = findLane(perImg)

    ## 2.5 Result
    result = drawResult(undis_img, Minv, perImg, ploty, Lfit, Rfit)

    font = cv2.FONT_HERSHEY_SIMPLEX
    cv2.putText( result, "Radius Left [m]: " + str(Curature_left), (70, 100),font, 1, (255,0,0), 2, cv2.LINE_AA )    
    cv2.putText( result, "Radius Right[m]: " + str(Curature_right), (70, 150),font, 1, (255,0,0), 2, cv2.LINE_AA )    
    cv2.putText( result, "Offset [m]: " + str(offset_m), (70, 200),font, 1, (255,0,0), 2, cv2.LINE_AA )


    # 3. Result save
    foldname = fname[0:14] +  fname[14:int(len(fname)-4)] + fname[13]
    isFolder = os.path.isdir(foldname)

    if isFolder == False:
        os.mkdir(foldname)    

    cv2.imwrite(foldname + '1_raw_img.jpg' , img )
    cv2.imwrite(foldname + '2_undistortion.jpg' , undis_img )
    cv2.imwrite(foldname + '3_filter_r.jpg' , combined_g*255)
    cv2.imwrite(foldname + '4_filter_s.jpg' , combined_s*255)
    cv2.imwrite(foldname + '5_combined.jpg' , combined*255)
    cv2.imwrite(foldname + '6_perspective.jpg' , perImg*255 )
    cv2.imwrite(foldname + '7_lanefinding.jpg' , out )
    cv2.imwrite(foldname + '8_result.jpg' , result )



C:\Users\Lim\Anaconda3\envs\udacity\lib\site-packages\ipykernel\__main__.py:131: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the future


In [12]:
from moviepy.editor import VideoFileClip
from IPython.display import HTML

filename = "challenge_video.mp4"

video_output = './test_video/result_' + filename
clip = VideoFileClip( "./test_video/" + filename )

# output_clip = clip.fl_image( process_image )
# %time output_clip.write_videofile(video_output, audio=False)
# del clip

C:\Users\Lim\Anaconda3\envs\udacity\lib\site-packages\skimage\filter\__init__.py:6: skimage_deprecation: The `skimage.filter` module has been renamed to `skimage.filters`.  This placeholder module will be removed in v0.13.
  warn(skimage_deprecation('The `skimage.filter` module has been renamed '
C:\Users\Lim\Anaconda3\envs\udacity\lib\site-packages\moviepy\audio\io\readers.py:110: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the future
  self.nchannels))


In [21]:
time = 13
Challenge_image = clip.get_frame(time)
cv2.imwrite("./chanllenge_images/challenge_" + str(time) + ".jpg",  cv2.cvtColor(Challenge_image, cv2.COLOR_RGB2BGR))



True

# Challenge image

In [39]:
import os

images = glob.glob('./chanllenge_images/*.jpg')

for fname in images:    
# fname = './test_images/test1.jpg'

    # 1. Load a test image
    img = cv2.imread(fname)
    

    # 2. Pipeline for lane finding
    ## 2.1 Undistortion
    undis_img = undistortImg( img, mtx, dist )

    # 2.2 Filtering using gradient (sobel)
    img_hls =  cv2.cvtColor(undis_img, cv2.COLOR_RGB2HLS)
    
    gray = undis_img[:,:,0]
    
    img_r = undis_img[:,:,0]
    img_g = undis_img[:,:,1]
    img_b = undis_img[:,:,2]
    
    img_s = img_hls[:,:,2]
    img_h = img_hls[:,:,0]
    img_l = img_hls[:,:,1]
        
    
    combined_sobel_g, combined_dir_g, combined_g = lanefilter_angle(gray, 
                                                             Ksizex=15, threshx=(5,255), 
                                                             Ksizey=15, threshy=(50,255),                                                              
                                                             ksize_dir=15, thresh_dir=(0.8, 1.4))
    
    combined_sobel_s, combined_dir_s, combined_s = lanefilter(img_s, 
                                                             Ksizex=15, threshx=(30,255), 
                                                             Ksizey=15, threshy=(80,255), 
                                                             Ksize_mag=15, thresh_mag=(40,255), 
                                                             ksize_dir=15, thresh_dir=(0.8, 1.4))
    
    combined_sobel_l, combined_dir_l, combined_l = lanefilter_angle(img_l, 
                                                         Ksizex=15, threshx=(10,255), 
                                                         Ksizey=15, threshy=(30,255),                                                          
                                                         ksize_dir=15, thresh_dir=(0.8, 1.4))
    img_test = np.zeros_like(img_l)
    img_test[(img_h > 100)] = 1 
        
    combined = np.zeros_like(gray)
    combined[ (combined_g==1) | (combined_s==1) ] = 1


    ## 2.3 Perspective transform
    offset_x = 100
    offset_y = 100

    yframe = 480
    xframe_ls = 300
    xframe_rs = 958    
    xframe_ll = 600
    xframe_rl = 718
        
    perImgSize_w = 480
    perImgSize_h = 720

    src = np.float32( [[xframe_ll, yframe], [xframe_rl, yframe], [xframe_rs, 680], [xframe_ls, 680]] )
    dst = np.float32( [[offset_x,0], [perImgSize_w-offset_x,0], [perImgSize_w-offset_x,perImgSize_h], [offset_x, perImgSize_h]])
    perImg, Minv = convBirdEyeView( combined, src, dst )    

    ## 2.4 Find lanes in a test image
    ## ( with Calculate the radius of curvature of lane and the position of the vehicle )
    out, ploty, Lfit, Rfit, Curature_left,  Curature_right, offset_m = findLane(perImg)

    ## 2.5 Result
    result = drawResult(undis_img, Minv, perImg, ploty, Lfit, Rfit)

    font = cv2.FONT_HERSHEY_SIMPLEX
    cv2.putText( result, "Radius Left [m]: " + str(Curature_left), (70, 100),font, 1, (255,0,0), 2, cv2.LINE_AA )    
    cv2.putText( result, "Radius Right[m]: " + str(Curature_right), (70, 150),font, 1, (255,0,0), 2, cv2.LINE_AA )    
    cv2.putText( result, "Offset [m]: " + str(offset_m), (70, 200),font, 1, (255,0,0), 2, cv2.LINE_AA )


    # 3. Result save
    foldname = fname[0:19] +  fname[19:int(len(fname)-4)] + fname[19]
    isFolder = os.path.isdir(foldname)

    if isFolder == False:
        os.mkdir(foldname)    

    cv2.imwrite(foldname + '1_raw_img.jpg' , img )
    cv2.imwrite(foldname + '2_undistortion.jpg' , undis_img )
    cv2.imwrite(foldname + '3_filter_r.jpg' , combined_g*255)
    cv2.imwrite(foldname + '4_filter_s.jpg' , combined_s*255)
    cv2.imwrite(foldname + '5_combined.jpg' , combined*255)
    cv2.imwrite(foldname + '6_perspective.jpg' , perImg*255 )
    cv2.imwrite(foldname + '7_lanefinding.jpg' , out )
    cv2.imwrite(foldname + '8_result.jpg' , result_img )
    

C:\Users\Lim\Anaconda3\envs\udacity\lib\site-packages\ipykernel\__main__.py:131: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the future
